In [ ]:
from utils.parse_openapi import parse_openapi_tools
import json

# 假设 openapi.json 已经保存到本地
with open("openapi.json", "r", encoding="utf-8") as f:
    openapi_json = json.load(f)

base_url = "http://8.218.206.87:8500"
tools = parse_openapi_tools(openapi_json, base_url)

# 查看工具信息
for name, info in tools.items():
    print(f"工具名: {name}")
    print(f"描述: {info['description']}")
    print(f"参数: {info['parameters']}")
    print("-" * 50)


工具名: get_idea_by_need_get_idea_by_need_post
描述: Get Idea By Need
参数: {'need': {'type': 'string', 'description': 'Need', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 20}}
--------------------------------------------------
工具名: get_fun_by_idea_get_fun_by_idea_post
描述: 根据 idea 搜索对应的 fun
参数: {'idea': {'type': 'string', 'description': 'Idea', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 20}}
--------------------------------------------------
工具名: get_fun_detail_get_fun_detail_post
描述: 根据函数名获取函数详细信息，只返回指定字段
参数: {'name': {'type': 'string', 'description': 'Name', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 1}}
--------------------------------------------------
工具名: get_enum_detail_get_enum_detail_post
描述: 根据联合体名获取联合体详细信息，只返回指定字段
参数: {'name': {'type': 'string', 'description': 'Name', 'default': None}, 'limit': {'type': 'integer', 'description': 'Limit', 'default': 1}}
----------------

In [5]:
tool_name = "get_idea_by_need_get_idea_by_need_post"

if tool_name in tools:
    func = tools[tool_name]["func"]
    result = func(need="生成一个自动化脚本", limit=5)  # 参数按 JSON Schema 提供
    print("调用结果:", result)


调用结果: [{'need': '通过用户输入的表达式名称创建对应的智能标量表达式并显示结果', 'idea': '1.初始化NX开发环境\n2.提示用户输入表达式名称\n3.根据表达式名称获取表达式字符串\n4.解析表达式字符串获取表达式tag\n5.创建智能标量表达式\n6.将结果写入列表窗口\n7.终止NX开发环境'}, {'need': '通过用户输入的表达式名称创建对应的智能标量表达式并显示结果', 'idea': '1.初始化NX开发环境\n2.提示用户输入表达式名称\n3.根据表达式名称获取表达式字符串\n4.解析表达式字符串获取表达式tag\n5.创建智能标量表达式\n6.将结果写入列表窗口\n7.终止NX开发环境'}, {'need': '在所有图纸页面上批量创建包含指定文本的注释', 'idea': '1.初始化NX开发环境\n2.获取当前NX会话的显示状态\n3.获取当前工作部件中所有图纸的标识符数组\n4.遍历图纸数组并打开每个图纸\n5.若当前为单视图模式则切换至图纸工作模式\n6.在指定位置创建多行文本注释\n7.释放图纸数组内存\n8.终止NX开发环境'}, {'need': '实现自动标注功能：选择视图并将其添加到指定的部件清单中', 'idea': '1.初始化UF环境\n2.设置撤销标记点\n3.选择部件清单\n4.选择视图\n5.获取部件清单的表格注释\n6.查询部件清单的偏好设置\n7.设置部件清单的符号类型\n8.将视图添加到部件清单中\n9.更新部件清单\n10.终止UF环境'}, {'need': '实现自动标注功能：选择视图并将其添加到指定的部件清单中', 'idea': '1.初始化UF环境\n2.设置选择部件清单的过滤条件\n3.选择部件清单\n4.选择视图\n5.获取部件清单的表格注释\n6.获取部件清单的偏好设置\n7.设置部件清单的符号类型\n8.将视图添加到部件清单中\n9.更新部件清单\n10.终止UF环境'}]


In [1]:
from utils.openapi_to_tools_json import openapi_to_mcp_tools
# 示例 openapi.json
import json

# 假设 openapi.json 已经保存到本地
with open("openapi.json", "r", encoding="utf-8") as f:
    example_openapi = json.load(f)

mcp_tools = openapi_to_mcp_tools(example_openapi)
print(json.dumps(mcp_tools, indent=2, ensure_ascii=False))

{
  "tools": [
    {
      "name": "get_idea_by_need_get_idea_by_need_post",
      "description": "Get Idea By Need",
      "input_schema": {
        "type": "object",
        "properties": {
          "need": {
            "anyOf": [
              {
                "type": "string"
              },
              {
                "type": "null"
              }
            ],
            "title": "Need",
            "description": "Need"
          },
          "limit": {
            "type": "integer",
            "title": "Limit",
            "default": 20,
            "description": "Limit"
          }
        }
      }
    },
    {
      "name": "get_fun_by_idea_get_fun_by_idea_post",
      "description": "Get Fun By Idea",
      "input_schema": {
        "type": "object",
        "properties": {
          "idea": {
            "anyOf": [
              {
                "type": "string"
              },
              {
                "type": "null"
              }
            ],
   

In [ ]:
# test_register_tools_request.py
import requests

# FastAPI 服务地址
BASE_URL = "http://8.218.206.87:8500"

# 构造最小 OpenAPI JSON 示例

with open("openapi.json", "r", encoding="utf-8") as f:
    example_openapi = json.load(f)
payload = {
    "openapi_json": example_openapi,
    "base_url": BASE_URL
}

response = requests.post(f"{BASE_URL}/register_tools", json=payload)

print("状态码:", response.status_code)
print("返回内容:", response.json())


状态码: 200
返回内容: {'success': True, 'message': '成功注册 5 个工具到内存'}


In [13]:
import requests
import json

BASE_URL = "http://127.0.0.1:8000"

payload = {
    "task": "你说一下你手里的工具是什么，说明一下用法"
}

response = requests.post(f"{BASE_URL}/run_agent", json=payload)

print("状态码:", response.status_code)
try:
    print("返回数据:", json.dumps(response.json(), ensure_ascii=False, indent=2))
except Exception:
    print("返回内容不是 JSON:", response.text)


状态码: 200
返回数据: {
  "result": "我目前可以使用的工具有以下五个，每个工具的用途和参数如下：\n\n1. **get_idea_by_need_get_idea_by_need_post**\n- 用途：根据用户的需求（need）生成相关的产品或功能创意（idea）。\n- 参数：\n- `need`（字符串，必填）：描述用户的具体需求。\n- `limit`（整数，可选，默认20）：返回创意的数量上限。\n\n2. **get_fun_by_idea_get_fun_by_idea_post**\n- 用途：根据一个创意（idea）获取对应的功能点（fun）。\n- 参数：\n- `idea`（字符串，必填）：输入一个具体的创意描述。\n- `limit`（整数，可选，默认20）：返回功能点的数量上限。\n\n3. **get_fun_detail_get_fun_detail_post**\n- 用途：获取某个具体功能点的详细信息。\n- 参数：\n- `name`（字符串，必填）：功能点的名称。\n- `limit`（整数，可选，默认1）：返回结果数量上限。\n\n4. **get_enum_detail_get_enum_detail_post**\n- 用途：获取某个枚举类型（enum）的详细定义和取值。\n- 参数：\n- `name`（字符串，必填）：枚举类型的名称。\n- `limit`（整数，可选，默认1）：返回结果数量上限。\n\n5. **get_structure_detail_get_structure_detail_post**\n- 用途：获取某个数据结构（structure）的详细字段定义。\n- 参数：\n- `name`（字符串，必填）：数据结构的名称。\n- `limit`（整数，可选，默认10）：返回结果数量上限。\n\n这些工具可以帮助从用户需求出发，逐步生成创意、功能点，并查看具体的功能、枚举或结构的详细信息。"
}
